In [2]:
# Importing Libraries
import sys
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
import matplotlib as mt
print mt.__version__

2.2.3


In [12]:
#Loading the dataset
data = pd.read_csv("./data/creditcard.csv")

In [27]:
#Exploring the dataset
data = data.sample(frac = 0.1, random_state = 1)
print data.shape
data.hist(figsize = (20,20))
plt.show()

In [28]:
# Choosing which entries are fraud
fraud = data[data['Class'] == 1]
valid = data[data['Class'] == 0]
outlier_fraction = len(fraud) / float(len(valid))
print len(fraud)

49


In [29]:
# Getting all the columns from the dataset
columns = data.columns.tolist()

# Filtering unwanted columns
columns = [c for c in columns if c not in ["Class"]]

target = "Class"

# Preparing the actual dataset
x = data[columns]
y = data[target]

print x.shape
print y.shape

(28481, 30)
(28481,)


In [30]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

# Define a random state
state = 1

# Putting the classifiers into a map for making them easy to use
classifiers = {
    "IsolationForest" : IsolationForest(max_samples=len(x),
                                       contamination = outlier_fraction,
                                       random_state = state),
    "LocalOutlierFactor" : LocalOutlierFactor(
    n_neighbors = 20,
    contamination = outlier_fraction)
}

In [31]:
n_outliers = len(fraud)

for i, (clf_name, clf) in enumerate(classifiers.items()):
    # Choosing actions according to classifier
    if clf_name == "LocalOutlierFactor":
        y_pred = clf.fit_predict(x)
    else:
        clf.fit(x)
        scores_pred = clf.decision_function(x)
        y_pred = clf.predict(x)
        
    # Reshaping the prediction values
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    
    n_errors = (y_pred != y).sum()
    
    print("NUmber of errors in {} is: {}".format(clf_name,n_errors))
    print(accuracy_score(y, y_pred))
    print(classification_report(y, y_pred))

NUmber of errors in LocalOutlierFactor is: 97
0.996594220709
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.02      0.02      0.02        49

   micro avg       1.00      1.00      1.00     28481
   macro avg       0.51      0.51      0.51     28481
weighted avg       1.00      1.00      1.00     28481



/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/iforest.py:418: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


NUmber of errors in IsolationForest is: 71
0.997507110003
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.28      0.29      0.28        49

   micro avg       1.00      1.00      1.00     28481
   macro avg       0.64      0.64      0.64     28481
weighted avg       1.00      1.00      1.00     28481

